In [ ]:
from dkube.sdk import *
import os

## Initializing feilds

In [ ]:
user = os.getenv('USERNAME')
token = os.getenv('DKUBE_USER_ACCESS_TOKEN')
inp_path = "/opt/dkube/input"
out_path = "/opt/dkube/output"
code_name = "mnist-fs"
dataset_name = "mnist"
training_fs = "mnist-fs"
model_name = "mnist"
framework = "tensorflow_1.14"
image = "ocdr/d3-datascience-tf-cpu:v1.14-3"
preprocess_script = f"python featureset.py --fs {training_fs}"
training_script = f"python model.py --fs {training_fs}"
transformer_script = "tf/classification/mnist-fs/digits/transformer/transformer.py"

In [ ]:
# Dkube Client handle
api = DkubeApi(token=token)

## Creating Resources

In [ ]:
print(f"Adding code {code_name}")
code = DkubeCode(user, name=code_name)
code.update_git_details("https://github.com/oneconvergence/dkubeio-examples/tree/master/tf/classification/mnist-fs/digits/classifier/program")
try:
    api.create_code(code)
    print(f"Code {code_name} added")
except BaseException as e:
    print(e)

print(f"Adding dataset {dataset_name}")
dataset = DkubeDataset(user, name=dataset_name)
dataset.update_dataset_source(source='git')
dataset.update_git_details("https://github.com/oneconvergence/dkubeio-examples-data/tree/master/tf/mnist")
try:
    api.create_dataset(dataset)
    print(f"Dataset {dataset_name} added")
except BaseException as e:
    print(e)

print(f"Adding featureset {training_fs}")
featureset = DkubeFeatureSet(name=training_fs)
try:
    api.create_featureset(featureset)
    print(f"Featureset {training_fs} added")
except BaseException as e:
    print(e)
    
print(f"Adding model {model_name}")
model = DkubeModel(user, name=model_name)
model.update_model_source(source='dvs')
try:
    api.create_model(model)
    print(f"Model {model_name} added")
except BaseException as e:
    print(e)

## Required kfp imports and conponents load

In [ ]:
import kfp
import kfp.dsl as dsl
import kfp.compiler as compiler
from kubernetes import client as k8s_client
from kfp import components
import json
from random import randint

dkube_preprocess_op         = components.load_component_from_file("/mnt/dkube/pipeline/components/preprocess/component.yaml")
dkube_training_op           = components.load_component_from_file("/mnt/dkube/pipeline/components/training/component.yaml")
dkube_serving_op            = components.load_component_from_file("/mnt/dkube/pipeline/components/serving/component.yaml")

## Pipeline definition

In [ ]:
@kfp.dsl.pipeline(
    name='dkube-mnistfs-pl',
    description='sample mnist featureset pipeline with dkube components'
)
def mnistfs_pipeline(
    # Dkube user token
    token = token, 
    # Code repo name which is added on Dkube platform
    code_name = "mnist-fs",
    # MNIST dataset name
    dataset_name = "mnist",
    # Featureset name
    training_fs = "mnist-fs",
    # Model name
    model_name = "mnist"):
    preprocessing = dkube_preprocess_op(token, json.dumps({"image": image}),
                                program=code_name, run_script=preprocess_script,
                                datasets = json.dumps([str(dataset_name)]), 
                                input_dataset_mounts = json.dumps([inp_path]),
                                output_featuresets=json.dumps([str(training_fs)]), 
                                output_featureset_mounts=json.dumps([out_path])).set_display_name("MNIST Preprocessing")
    
    training      = dkube_training_op(token, json.dumps({"image": image}),
                            framework="tensorflow", version="1.14",
                            program=code_name, run_script=training_script,
                            featuresets=json.dumps([str(training_fs)]), outputs=json.dumps([str(model_name)]),
                            input_featureset_mounts=json.dumps([inp_path]),
                            output_mounts=json.dumps([out_path])).after(preprocessing).set_display_name("MNIST Training")
    
    serving       = dkube_serving_op(token, training.outputs['artifact'],
                            device="cpu", serving_image=json.dumps({"image": "ocdr/tensorflowserver:1.14"}),
                            transformer_image=json.dumps({"image": image}),
                            transformer_project=code_name,
                            transformer_code=transformer_script).after(training).set_display_name("MNIST Serving")

## Creating experiment

In [ ]:
experiment_name = 'Dkube- MNIST-FS pl'
client = kfp.Client(existing_token=token)
mnist_experiment = client.create_experiment(name=experiment_name)

## Compiling and uploading pipeline(make sure the pipeline name is unique)

In [ ]:
arguments = {"token":token}
compiler.Compiler().compile(mnistfs_pipeline, "mnistfs-pipeline.zip")
try:
    pipeline = client.upload_pipeline("mnistfs-pipeline.zip", pipeline_name = "mnist-fs-pipeline")
except BaseException as e:
    print(e)
runid = 1

## Creating run from pipeline

In [ ]:
run = client.run_pipeline(mnist_experiment.id, job_name="[MNIST] Run" + str(runid), pipeline_id=pipeline.id, params=arguments)
runid += 1